In [1]:
%matplotlib inline

import os

# let's not pollute this blog post with warnings
from warnings import filterwarnings
filterwarnings('ignore')

import keras
import numpy as np
import pandas as pd
import skvideo.io as skv
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# load the data
run = '-VGG-micro'
labelpath = os.path.join('train_labels.csv')
train_labels = pd.read_csv(labelpath, index_col='filename')

In [3]:
train_labels.head()

,bird,blank,cattle,chimpanzee,elephant,forest buffalo,gorilla,hippopotamus,human,hyena,...,other (primate),pangolin,porcupine,reptile,rodent,small antelope,small cat,wild dog,duiker,hog
filename,,,,,,,,,,,,,,,,,,,,,
000libDc84.mp4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003TeGtbkD.mp4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006jFoesFi.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
008uxqP8IN.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0094UxdyyZ.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204130 entries, 000libDc84.mp4 to zzzu2lK8bC.mp4
Data columns (total 24 columns):
bird                   204130 non-null float64
blank                  204130 non-null float64
cattle                 204130 non-null float64
chimpanzee             204130 non-null float64
elephant               204130 non-null float64
forest buffalo         204130 non-null float64
gorilla                204130 non-null float64
hippopotamus           204130 non-null float64
human                  204130 non-null float64
hyena                  204130 non-null float64
large ungulate         204130 non-null float64
leopard                204130 non-null float64
lion                   204130 non-null float64
other (non-primate)    204130 non-null float64
other (primate)        204130 non-null float64
pangolin               204130 non-null float64
porcupine              204130 non-null float64
reptile                204130 non-null float64
rodent                 2041

In [5]:
train_labels.sum(axis=0).sort_values(ascending=False)

blank                  122270.0
duiker                  21601.0
other (primate)         20453.0
human                   20034.0
chimpanzee               5045.0
hog                      4650.0
rodent                   2911.0
bird                     2386.0
other (non-primate)      1883.0
elephant                 1085.0
porcupine                 569.0
cattle                    372.0
small antelope            273.0
large ungulate            224.0
leopard                   209.0
hippopotamus              175.0
gorilla                   174.0
small cat                  79.0
pangolin                   63.0
wild dog                   21.0
hyena                      10.0
forest buffalo              9.0
reptile                     8.0
lion                        2.0
dtype: float64

In [6]:
(train_labels.sum(axis=1) > 1).sum()

374

In [ ]:
from primatrix_dataset_utils import Dataset

In [ ]:
datapath = os.path.join('.')
whichset = run.split('-')[-1]
redframes = whichset == 'nano'
data = Dataset(datapath=datapath, 
               dataset_type=whichset,
               reduce_frames=redframes, 
               batch_size=32, 
               test=False)

In [ ]:
data.num_classes

In [ ]:
data.num_frames

In [ ]:
data.batch_size

In [ ]:
data.num_samples

In [ ]:
data.y_train.shape[0]

In [ ]:
data.y_val.shape[0]

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

In [ ]:
from keras.models import Sequential
from keras.layers import TimeDistributed, GlobalAveragePooling2D, Activation, Dense, Input
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dropout, Flatten
from keras.layers import concatenate
from keras import regularizers
from keras import initializers
from keras import constraints
from keras.models import Model
# Backend
from keras import backend as K
# Utils
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file

def Conv2D_bn(x,
              filters,
              filter_size,
              padding='same',
              strides=(1, 1),
              name=None,
             activation='relu'):
    num_row, num_col = filter_size
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = -1
    x = TimeDistributed(Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name,
        kernel_initializer=initializers.he_normal()))(x)
    x = TimeDistributed(BatchNormalization(axis=bn_axis, scale=False, name=bn_name))(x)
    x = TimeDistributed(Activation('relu', name=name))(x)
    return x


def VGG19(include_top=True,
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=24):
    channel_axis = -1
    inputs = Input((data.num_frames, data.width, data.height, 3))
    x = BatchNormalization(axis=-1)(inputs)
    
    x = Conv2D_bn(x, 32, (3, 3), activation='relu', padding='same', name='block1_conv1')
    x = Conv2D_bn(x, 32, (3, 3), activation='relu', padding='same', name='block1_conv2')
    x = TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))(x)

    # Block 2
    x = Conv2D_bn(x, 64, (3, 3), activation='relu', padding='same', name='block2_conv1')
    x = Conv2D_bn(x, 64, (3, 3), activation='relu', padding='same', name='block2_conv2')
    x = TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))(x)

    # Block 3
    x = Conv2D_bn(x, 128, (3, 3), activation='relu', padding='same', name='block3_conv1')
    x = Conv2D_bn(x, 128, (3, 3), activation='relu', padding='same', name='block3_conv2')
    x = Conv2D_bn(x, 128, (3, 3), activation='relu', padding='same', name='block3_conv3')
    x = Conv2D_bn(x, 128, (3, 3), activation='relu', padding='same', name='block3_conv4')
    x = TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))(x)

    # Block 4
    x = Conv2D_bn(x, 256, (3, 3), activation='relu', padding='same', name='block4_conv1')
    x = Conv2D_bn(x, 256, (3, 3), activation='relu', padding='same', name='block4_conv2')
    x = Conv2D_bn(x, 256, (3, 3), activation='relu', padding='same', name='block4_conv3')
    x = Conv2D_bn(x, 256, (3, 3), activation='relu', padding='same', name='block4_conv4')
    x = TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))(x)

    # Block 5
    x = Conv2D_bn(x, 512, (3, 3), activation='relu', padding='same', name='block5_conv1')
    x = Conv2D_bn(x, 512, (3, 3), activation='relu', padding='same', name='block5_conv2')
    x = Conv2D_bn(x, 512, (3, 3), activation='relu', padding='same', name='block5_conv3')
    x = Conv2D_bn(x, 512, (3, 3), activation='relu', padding='same', name='block5_conv4')
    x = TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))(x)

    if include_top:
        # Classification block
        x = TimeDistributed(Flatten(name='flatten'))(x)
        x = TimeDistributed(Dense(4096, activation='relu', name='fc1'))(x)
        x = TimeDistributed(Dense(4096, activation='relu', name='fc2'))(x)   
        x = LSTM(256, return_sequences=False)(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)

    model = Model(inputs, x, name='vgg16')

    # load weights
    return model

In [ ]:
# instantiate model
model = VGG19()


# classifier with sigmoid activation for multilabel

adam = keras.optimizers.Adam(lr=0.001, decay=0.0, clipnorm=5.)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.00001, verbose=True)


# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, reduce_lr]


# look at the params before training
model.summary()

In [ ]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=int(data.num_batches/20),                  # data.num_batches to train on full set 
    epochs=6000, 
    validation_data=data.val_batches(), 
    validation_steps=int(data.num_val_batches/20),                  # data.num_val_batches to validate on full set
    callbacks=callbacks_list
)

In [ ]:
# load model
from keras.models import load_model

trained_model = load_model(model_name)

# generate predictions
for batch_num in tqdm(range(data.num_test_batches), total=data.num_test_batches):

    # make predictions on batch
    results = trained_model.predict(next(data.test_batches()), 
                                          batch_size=data.batch_size, 
                                          verbose=0)

    # update submission format dataframe stored in dataset object
    data.update_predictions(results)          
    

In [ ]:
data.predictions.to_csv(os.path.join(data.datapath, 'predictions' + run + '.csv'))